# 爬取视频
一般情况下视频包装在`<video src='xxx.mp4'></video>`。

一般不会有网站直接把一个视频嵌入到网页里面，第一是数据量太大，加载很慢，用户很不爽；另外就是加载一次视频的流量太大，如果有的用户不是非得从头看到尾，那么其他的地方就白加载了，这也是一种浪费。

## 一般的视频网站是怎么做的？
用户上传->转码(把视频进行处理：2K，1080，标清)->对视频进行切片处理(把单个的文件进行拆分)：用户在拉动进度条的时候，只加载该位置的切片，然后在读的时候逐步地加载后面的切片。

由于分成的切片肯定不能随便堆在存储区，所以需要一个文件来记录：1.视频播放的顺序；2.视频存放的路径。

我们一般倾向于把切分好的视频数据放到一个叫做`M3U`的**文本**。然后对`M3U`文件进行utf-8编码，就得到了`M3U8`文件，该文件的结构大概如下所示。

![M3U8](m3u8.png)

以井号`#`开头的数据都有一些自己的含义。在这个文件中要关注的地方：

1. 井号开头的部分：加密方法的了解。
2. .ts文件：反映的是每一个切片的URL。

## 怎样抓取一个视频？
1. 找到m3u8文件(各种手段)；
2. 通过m3u8下载到ts文件；
3. 通过各种手段(不限于编程手段)把ts文件合并为一个mp4文件。

In [ ]:
import requests
import re
url='https://www.hhedai.com/vod/63046-1-1.html'
resp=requests.get(url).text
# print(resp.text)
obj=re.compile(r'<script>var cms_player = {"yun":true,"url":"(?P<url>.*?)"',re.S)
url=re.search(obj,resp).group('url').replace("\\","")
# print(url)
m3u81=requests.get(url).text
# print(m3u81)
obj2=re.compile(r'(?<=\n)(.*)',re.S)
extend=re.search(obj2,m3u81).group(1)
extend=re.search(obj2,extend).group(1)
# print(extend)
# print('done!')
url=url.rsplit('/',1)[0]+'/'+extend
print(url)
# if url=='https://pptv.sd-play.com/202306/05/GMCCZdL3m83/video/900k_0X480_64k_25/hls/index.m3u8':
    #      https://pptv.sd-play.com/202306/05/GMCCZdL3m83/video/900k_0X480_64k_25/hls/index.m3u8
resp1=requests.get('https://pptv.sd-play.com/202306/05/GMCCZdL3m83/video/900k_0X480_64k_25/hls/index.m3u8')  
print(resp1.text)
with open('test.m3u8','wb')as f:
    f.write(resp1.content)
f.close()
print('下载完毕！')
# else:
#     print('url不匹配！')
# https://pptv.sd-play.com/202306/05/GMCCZdL3m83/video/900k_0X480_64k_25/hls/index.m3u8
# https://pptv.sd-play.com/202306/05/GMCCZdL3m83/video/900k_0X480_64k_25/hls/index.m3u8

In [7]:
n=1
with open('test.m3u8','r',encoding='utf-8')as f:
    for line in f:
        line=line.strip()
        if line.startswith('#'):#判断一个字符串以#打头
            continue
        # print(line)
        resp3=requests.get(line)
        f1=open(f'video/{n}.ts',mode='wb')
        f1.write(resp3.content)
        print(n,'.ts done!')
        n+=1
    f.close()
    resp3.close()

        

1 .ts done!
2 .ts done!
3 .ts done!
4 .ts done!
5 .ts done!
6 .ts done!
7 .ts done!
8 .ts done!
9 .ts done!
10 .ts done!
11 .ts done!
12 .ts done!


: 

: 